In [1]:
import pandas as pd
from urllib import request

In [30]:
YEARS = ['2010', '2011', '2012', '2013', '2014', '2015']
expenses = {}

wikipedia_page = "https://en.wikipedia.org/wiki/List_of_United_Kingdom_Parliament_constituencies"
response = pd.read_html(wikipedia_page, header=0)
response[1] = response[1][response[1]['Country of the UK'] == "England"]
response[1].drop([i for i in response[1].columns if i not in ['Constituency','Region']], axis = 1, inplace = True)

In [31]:
YEARS = ['2010', '2011', '2012', '2013', '2014', '2015']
expenses = {}
for year in YEARS:
    expenses.update({str(year) : pd.read_csv("../Rytch saves the day/expenses_GOR_" + year+".csv",  low_memory=False)})

In [42]:
allData = pd.DataFrame(columns=["MP's Name", 
                              "MP's Constituency",
                              'Category',
                              'Amount Paid',
                              'Year'])
holding = {}

In [43]:
for year, data in expenses.items():
    data['Year'] = year
    data.rename(columns={col : col.strip().replace("\\","") for col in data.columns}, inplace=True)
    data.drop([col for col in data.columns if col not in ["MP's Name", 
                                                          "MP's Constituency",
                                                          'Category',
                                                          'Amount Paid',
                                                          'Year']], axis = 1, inplace=True)
    #data.dropna(axis = 0, subset=['Region'], inplace=True)
    data.reset_index(inplace=True, drop=True)
    allData = pd.concat([allData, data], axis = 0)    

In [45]:
allData["Constituency"] = [con[:-3] if con[-3:] != 'own' else con for con in allData["MP's Constituency"]]
allData.drop(["MP's Constituency"], axis = 1, inplace = True)

In [48]:
allDataReg = allData.merge(response[1], how = 'outer', on='Constituency')
allDataReg.head()

,MP's Name,Category,Amount Paid,Year,Constituency,Region
0,Ken Clarke,Travel,57.6,2013,Rushcliffe,East Midlands
1,Ken Clarke,Travel,57.6,2013,Rushcliffe,East Midlands
2,Ken Clarke,Office Costs,577.7,2013,Rushcliffe,East Midlands
3,Ken Clarke,Office Costs,300.0,2013,Rushcliffe,East Midlands
4,Ken Clarke,Travel,57.6,2013,Rushcliffe,East Midlands


# Checking merge

In [58]:
allDataReg.loc[allDataReg['Region'].isnull(), 'Constituency'].unique()

array(['South Down'], dtype=object)

In [66]:
allDataReg.dropna(axis = 0, how = 'any', subset = ['Region'], inplace = True)
allDataReg.loc[allDataReg['Region'].isnull(), 'Constituency'].unique()

array([], dtype=object)

In [67]:
grouped = pd.DataFrame(allDataReg.groupby(['Year',  
                                        'Region',
                                        "Constituency",
                                        "MP's Name", 
                                        'Category', ], as_index=False)['Amount Paid'].sum().round(0))
grouped.rename(columns={"MP's Name" : "Name", "Amount Paid" : "Paid"}, inplace=True)


In [68]:
grouped.head()

,Year,Region,Constituency,Name,Category,Paid
0,2010,East Midlands,Amber Valley,Nigel Mills,Accommodation,8452.0
1,2010,East Midlands,Amber Valley,Nigel Mills,Constituency Rental,7997.0
2,2010,East Midlands,Amber Valley,Nigel Mills,General Admin,5243.0
3,2010,East Midlands,Amber Valley,Nigel Mills,Staffing,3878.0
4,2010,East Midlands,Amber Valley,Nigel Mills,Travel,4269.0


# Attaching constituency codes to df

In [69]:
cons = pd.read_json("con_to_reg.json", orient="index")
cons['concode'] = cons.index
cons.reset_index(drop = True, inplace = True)
cons.head()

,name,reg,concode
0,Aldershot,E15000008,E14000530
1,Aldridge-Brownhills,E15000005,E14000531
2,Altrincham and Sale West,E15000002,E14000532
3,Amber Valley,E15000004,E14000533
4,Arundel and South Downs,E15000008,E14000534


In [70]:
grouped.head()

,Year,Region,Constituency,Name,Category,Paid
0,2010,East Midlands,Amber Valley,Nigel Mills,Accommodation,8452.0
1,2010,East Midlands,Amber Valley,Nigel Mills,Constituency Rental,7997.0
2,2010,East Midlands,Amber Valley,Nigel Mills,General Admin,5243.0
3,2010,East Midlands,Amber Valley,Nigel Mills,Staffing,3878.0
4,2010,East Midlands,Amber Valley,Nigel Mills,Travel,4269.0


In [81]:
concodes = grouped.merge(cons, how='outer', left_on = 'Constituency', right_on = 'name')
concodes.head()

,Year,Region,Constituency,Name,Category,Paid,name,reg,concode
0,2010,East Midlands,Amber Valley,Nigel Mills,Accommodation,8452.0,Amber Valley,E15000004,E14000533
1,2010,East Midlands,Amber Valley,Nigel Mills,Constituency Rental,7997.0,Amber Valley,E15000004,E14000533
2,2010,East Midlands,Amber Valley,Nigel Mills,General Admin,5243.0,Amber Valley,E15000004,E14000533
3,2010,East Midlands,Amber Valley,Nigel Mills,Staffing,3878.0,Amber Valley,E15000004,E14000533
4,2010,East Midlands,Amber Valley,Nigel Mills,Travel,4269.0,Amber Valley,E15000004,E14000533


## Check which constituencies haven't merged properly - only Welsh ones

In [72]:
NonMergesGrouped = concodes[(concodes['Year'].isnull()) & (concodes['reg'].notnull()) ]
NonMergesGrouped

,Year,Region,Constituency,Name,Category,Paid,name,reg,concode
13265,NaN,NaN,NaN,NaN,NaN,NaN,Ynys Mśn,W08000001,W07000041
13266,NaN,NaN,NaN,NaN,NaN,NaN,Delyn,W08000001,W07000042
13267,NaN,NaN,NaN,NaN,NaN,NaN,Alyn and Deeside,W08000001,W07000043
13268,NaN,NaN,NaN,NaN,NaN,NaN,Wrexham,W08000001,W07000044
13269,NaN,NaN,NaN,NaN,NaN,NaN,Llanelli,W08000001,W07000045
13270,NaN,NaN,NaN,NaN,NaN,NaN,Gower,W08000001,W07000046
13271,NaN,NaN,NaN,NaN,NaN,NaN,Swansea West,W08000001,W07000047
13272,NaN,NaN,NaN,NaN,NaN,NaN,Swansea East,W08000001,W07000048
13273,NaN,NaN,NaN,NaN,NaN,NaN,Aberavon,W08000001,W07000049
13274,NaN,NaN,NaN,NaN,NaN,NaN,Cardiff Central,W08000001,W07000050


In [73]:
NonMergesCons = concodes[(concodes['Year'].notnull()) & (concodes['reg'].isnull()) ]
NonMergesCons

,Year,Region,Constituency,Name,Category,Paid,name,reg,concode


In [76]:
check = concodes[(concodes['Year'].isnull()) ]
check

,Year,Region,Constituency,Name,Category,Paid,name,reg,concode
13265,NaN,NaN,NaN,NaN,NaN,NaN,Ynys Mśn,W08000001,W07000041
13266,NaN,NaN,NaN,NaN,NaN,NaN,Delyn,W08000001,W07000042
13267,NaN,NaN,NaN,NaN,NaN,NaN,Alyn and Deeside,W08000001,W07000043
13268,NaN,NaN,NaN,NaN,NaN,NaN,Wrexham,W08000001,W07000044
13269,NaN,NaN,NaN,NaN,NaN,NaN,Llanelli,W08000001,W07000045
13270,NaN,NaN,NaN,NaN,NaN,NaN,Gower,W08000001,W07000046
13271,NaN,NaN,NaN,NaN,NaN,NaN,Swansea West,W08000001,W07000047
13272,NaN,NaN,NaN,NaN,NaN,NaN,Swansea East,W08000001,W07000048
13273,NaN,NaN,NaN,NaN,NaN,NaN,Aberavon,W08000001,W07000049
13274,NaN,NaN,NaN,NaN,NaN,NaN,Cardiff Central,W08000001,W07000050


# Getting rid of the welsh

In [82]:
concodes.dropna(axis = 0, how = 'any', inplace = True, subset = ['Paid'])
concodes[(concodes['Year'].isnull()) ]

,Year,Region,Constituency,Name,Category,Paid,name,reg,concode


In [83]:
concodes.drop(['Region','Constituency','name','reg'], axis = 1, inplace = True)
concodes.rename(columns={'concode' : 'ConstCode'}, inplace = True)
concodes.head()

,Year,Name,Category,Paid,ConstCode
0,2010,Nigel Mills,Accommodation,8452.0,E14000533
1,2010,Nigel Mills,Constituency Rental,7997.0,E14000533
2,2010,Nigel Mills,General Admin,5243.0,E14000533
3,2010,Nigel Mills,Staffing,3878.0,E14000533
4,2010,Nigel Mills,Travel,4269.0,E14000533


In [84]:
concodes.to_csv("ExpensesByRegion.csv")

In [ ]:
consts = grouped["Constituency"].unique()
suffixes = pd.Series([i[-3:] for i in consts])
suffixes.unique()

In [ ]:
invest = [i for i in consts if i[-3:] == 'own']
invest